<a href="https://colab.research.google.com/github/kghilothia06/Time-Series-Forecasting/blob/main/04_bidirectional_lstm_univariate_weekly_car_sales_9_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# univariate bidirectional lstm example for weekly sales prediction

Import Libraries

In [1]:
import pandas as pd
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional


Function to split a univariate sequence

In [2]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

read dataset in pandas

In [3]:
df = pd.read_excel('/content/Sales-weekly.xlsx',header=0,index_col=0)

Data Preparation for Model Training

In [4]:
# choose a number of time steps
n_steps = 3

In [5]:
#Convert 'Sales' column to a list
raw_seq=[]

for i in df['Sales']:
  raw_seq.append(i)

In [6]:
# split into samples
X, y = split_sequence(raw_seq, n_steps)

In [7]:
# taking all the data except that of 53rd week of December 2020 as training data
X_train = X[:len(X)-1]
y_train = y[:len(y)-1]

In [8]:
# test sample would be data corresponding to 53rd week of December 2020
X_test = X[len(X)-1]
y_test = y[len(y)-1] 

In [9]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))

Define Model

In [10]:
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [11]:
# fit model
model.fit(X_train, y_train, epochs=200, verbose=0)

Make Prediction

In [12]:
X_test = X_test.reshape((1, n_steps, n_features))

In [13]:
yhat = model.predict(X_test, verbose=0)
print('predicted value : ' + str(yhat[0][0]))
print('actual value : ' + str(y_test))

predicted value : 78215.4
actual value : 185158.73
